In [2]:
# realsense to point cloud test 1
import open3d as o3d
import matplotlib.pyplot as plt
import os
import cv2
folders=['color_pic', 'depth_pic', 'dColorMap']
pic_name_color=["color_1718275758.jpg"]
pic_name_depth=["depth_1718275758.jpg"]
path_color=os.path.join(folders[0],pic_name_color[0])
path_depth=os.path.join(folders[1],pic_name_depth[0])
# print("Read Redwood dataset")
color_raw = o3d.io.read_image(path_color)
depth_raw = o3d.io.read_image(path_depth)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_raw, depth_raw)
print(rgbd_image)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
RGBDImage of size 
Color image : 640x480, with 1 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.


In [ ]:
plt.subplot(1,2,1)
plt.title('cube grayscale image')
plt.imshow(rgbd_image.color)
plt.subplot(1,2,2)
plt.title('cube depth image')
plt.imshow(rgbd_image.depth)
plt.show()

In [ ]:

color_image = cv2.imread(path_color)
depth_image = cv2.imread(path_depth, cv2.IMREAD_UNCHANGED)



color_image_o3d = o3d.geometry.Image(color_image)

depth_image_o3d = o3d.geometry.Image(depth_image)


rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_image_o3d, 
    depth_image_o3d, 
    convert_rgb_to_intensity=False
)


width, height = color_image.shape[1], color_image.shape[0]
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx=500, fy=500, cx=width / 2, cy=height / 2)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic
)

# Flip the point cloud (to match Open3D's coordinate system)
pcd.transform([[1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, 1]])

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud")


o3d.io.write_point_cloud("cylinder_raw.ply", pcd)

In [ ]:
pcd=o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,o3d.camera.PinholeCameraIntrinsic(
    o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[1,0,0,1]])
o3d.visualization.draw_geometries([pcd],zoom=0.5)

In [ ]:
import cv2
color_image_path="color_1717674720.jpg"
depth_image_path="depth_1717674720.jpg"
color_image = cv2.imread(color_image_path)
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

color_image_o3d = o3d.geometry.Image(color_image)


depth_image_o3d = o3d.geometry.Image(depth_image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_image_o3d, 
    depth_image_o3d, 
    convert_rgb_to_intensity=False
)


width, height = color_image.shape[1], color_image.shape[0]
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx=500, fy=500, cx=width / 2, cy=height / 2)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsic
)

# Flip the point cloud (to match Open3D's coordinate system)
pcd.transform([[1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, -1, 0],
               [0, 0, 0, 1]])

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud")

# Optionally, save the point cloud to a file
o3d.io.write_point_cloud("output_point_cloud.ply", pcd)

In [ ]:

#Generate cone pcd


import numpy as np
import open3d as o3d

def generate_cone(base_radius, height, num_points):

    theta = np.linspace(0, 2 * np.pi, num_points)
    x_base = base_radius * np.cos(theta)
    y_base = base_radius * np.sin(theta)
    z_base = np.zeros(num_points)

    tapering_factor = np.linspace(0, 1, num_points)


    radii = base_radius * (1 - tapering_factor)

    points = []
    for i in range(num_points):
        points.extend(list(zip(x_base * radii[i], y_base * radii[i], np.full_like(theta, i * height / num_points))))


    apex = np.array([0, 0, height])
    points.append(apex)
    
    return np.array(points)

base_radius = 1
height = 2
num_points = 100
cone_points = generate_cone(base_radius, height, num_points)


pcd_cone = o3d.geometry.PointCloud()
pcd_cone.points = o3d.utility.Vector3dVector(cone_points)


o3d.io.write_point_cloud("cone.pcd", pcd_cone)


In [ ]:

def generate_cuboid(width, height, length, num_points_per_face):
    # Define the vertices of the cuboid
    vertices = np.array([
        [0, 0, 0],  # Front bottom left
        [width, 0, 0],  # Front bottom right
        [width, height, 0],  # Front top right
        [0, height, 0],  # Front top left
        [0, 0, length],  # Back bottom left
        [width, 0, length],  # Back bottom right
        [width, height, length],  # Back top right
        [0, height, length]  # Back top left
    ])

    # Define the indices of the cuboid's faces
    indices = [
        [0, 1, 2, 3],  # Front face
        [1, 5, 6, 2],  # Right face
        [5, 4, 7, 6],  # Back face
        [4, 0, 3, 7],  # Left face
        [3, 2, 6, 7],  # Top face
        [0, 4, 5, 1]   # Bottom face
    ]

    # Stack the vertices to form the cuboid
    points = []
    for face_indices in indices:
        face_points = vertices[face_indices]
        points.extend(face_points)

    # Add points on each face of the cuboid
    for i in range(num_points_per_face):
        # Points on the front face
        points.append([np.random.uniform(0, width), np.random.uniform(0, height), 0])
        # Points on the back face
        points.append([np.random.uniform(0, width), np.random.uniform(0, height), length])
        # Points on the left face
        points.append([0, np.random.uniform(0, height), np.random.uniform(0, length)])
        # Points on the right face
        points.append([width, np.random.uniform(0, height), np.random.uniform(0, length)])
        # Points on the top face
        points.append([np.random.uniform(0, width), height, np.random.uniform(0, length)])
        # Points on the bottom face
        points.append([np.random.uniform(0, width), 0, np.random.uniform(0, length)])

    return np.array(points)

# Generate cuboid points
width = 1
height = 1
length = 2
num_points_per_face = 400
cuboid_points = generate_cuboid(width, height, length, num_points_per_face)

# Create Open3D PointCloud object
pcd_cuboid = o3d.geometry.PointCloud()
pcd_cuboid.points = o3d.utility.Vector3dVector(cuboid_points)

# Save PointCloud to PCD file
o3d.io.write_point_cloud("cuboid.pcd", pcd_cuboid)


In [ ]:
def generate_cylinder(radius, height, num_points):
    # Generate points along the circumference
    theta = np.linspace(0, 2 * np.pi, num_points)
    x_circle = radius * np.cos(theta)
    y_circle = radius * np.sin(theta)

    # Generate points along the height
    z_height = np.linspace(0, height, num_points)

    # Stack the points to form the cylinder
    points = []
    for z in z_height:
        points.extend(list(zip(x_circle, y_circle, np.full_like(theta, z))))

    return np.array(points)

# Generate cylinder points
radius = 0.4
height = 2
num_points = 100
cylinder_points = generate_cylinder(radius, height, num_points)

# Create Open3D PointCloud object
pcd_cylinder = o3d.geometry.PointCloud()
pcd_cylinder.points = o3d.utility.Vector3dVector(cylinder_points)

# Save PointCloud to PCD file
o3d.io.write_point_cloud("cylinder.pcd", pcd_cylinder)


In [ ]:
def generate_sphere(radius, num_points):
    u = np.linspace(0, 2 * np.pi, num_points)
    v = np.linspace(0, np.pi, num_points)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
    return np.vstack((x.flatten(), y.flatten(), z.flatten())).T

# Generate sphere points
radius = 1
num_points = 100
points = generate_sphere(radius, num_points)

# Create Open3D PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Save PointCloud to PCD file
o3d.io.write_point_cloud("sphere.ply", pcd)



In [ ]:
pcd = o3d.io.read_point_cloud("sphere.pcd")

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Visualization")

In [ ]:

pcd = o3d.io.read_point_cloud("cone.pcd")

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Visualization")


In [3]:
pcd = o3d.io.read_point_cloud("cylinder.pcd")

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Visualization")

In [ ]:
pcd = o3d.io.read_point_cloud("cuboid.pcd")

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Visualization")

In [ ]:
pcd = o3d.io.read_point_cloud("output_point_cloud.ply")

o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Visualization")